In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import gmean
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve


conma=pd.DataFrame(np.array([[0,0],[0,0]]),columns=['예측0','예측1'],index=['실제0','실제1'])

In [2]:
def get_geoacc(estm_weights,gunT,budoT):
    #Threshold = Find_Threshold(estm_weights,gunT,budoT)
    Threshold = 0.5
    total = gunT.shape[0]+budoT.shape[0]
    proba = np.zeros((total,1))
    proba = np.array(proba)
    pred = np.zeros((total,1))
    pred = np.array(pred)
    real_gun = np.zeros((gunT.shape[0],1))
    real_budo = np.zeros((budoT.shape[0],1))
    real_budo = real_budo+1
    real = np.vstack([real_gun,real_budo])
    for i in range(gunT.shape[0]):
        for j in range(gunT.shape[1]):
            proba[i] = proba[i]+ gunT[i][j]*estm_weights[j]

    for i in range(budoT.shape[0]):
        for j in range(budoT.shape[1]):
            proba[gunT.shape[0]+i] = proba[gunT.shape[0]+i]+ budoT[i][j]* estm_weights[j]
    gun_miss_sum=0
    budo_miss_sum=0
    for i in range(budoT.shape[0]):
        if proba[i+gunT.shape[0]] <= Threshold:
            budo_miss_sum= budo_miss_sum +1
            pred[i+gunT.shape[0]] = 0
        else:
            pred[i+gunT.shape[0]] = 1
    for i in range(gunT.shape[0]):
        if proba[i] >= Threshold:
            gun_miss_sum = gun_miss_sum+1
            pred[i] = 1
    err = gun_miss_sum +budo_miss_sum
    results = confusion_matrix(y_true=real,y_pred=pred)
    df = pd.DataFrame(results,columns=['건전 예측','부도 예측'],index=['실제 건전','실제 부도'])
    conma_temp = makeconma(conma,results)
    geo_acc = geoacc(conma)
    return geo_acc

def makeconma(conma,mat):
    conma.iloc[0,0],conma.iloc[0,1],conma.iloc[1,0],conma.iloc[1,1]=mat[0,0],mat[0,1],mat[1,0],mat[1,1]
    return conma
def geoacc(mat):
    value = (mat.iloc[0,0] / sum(mat.iloc[0,:])) * (mat.iloc[1,1] / sum(mat.iloc[1,:]))
    return math.sqrt(value)

def get_proba(estm_weights,gunT,budoT):
    #Threshold = Find_Threshold(estm_weights,gunT,budoT)
    Threshold = 0.5
    total = gunT.shape[0]+budoT.shape[0]
    proba = np.zeros((total,1))
    proba = np.array(proba)
    pred = np.zeros((total,1))
    pred = np.array(pred)
    real_gun = np.zeros((gunT.shape[0],1))
    real_budo = np.zeros((budoT.shape[0],1))
    real_budo = real_budo+1
    real = np.vstack([real_gun,real_budo])
    for i in range(gunT.shape[0]):
        for j in range(gunT.shape[1]):
            proba[i] = proba[i]+ gunT[i][j]*estm_weights[j]

    for i in range(budoT.shape[0]):
        for j in range(budoT.shape[1]):
            proba[gunT.shape[0]+i] = proba[gunT.shape[0]+i]+ budoT[i][j]* estm_weights[j]
    return proba

def Find_Optimal_Cutoff(target, predicted):
    #print("target",target)
    #print("pred",predicted)
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr))
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index = i), 'threshold' : pd.Series(threshold, index = i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]
    return list(roc_t['threshold'])

def Find_Threshold(estm_weights,gunT,budoT):
    #print("gun:",gunT)
    #print("budo:",budoT)
    gunT=np.array(gunT)
    budoT=np.array(budoT)
    real_gun = np.zeros((gunT.shape[0],1))
    real_budo = np.zeros((budoT.shape[0],1))
    real_budo = real_budo+1
    real = np.vstack([real_gun,real_budo])
    proba = np.zeros((gunT.shape[0]+budoT.shape[0],1))
    proba = np.array(proba)
    for i in range(proba.shape[0]):
        proba[0] = 0
    #print(gunT.shape)
    #print("estm",estm_weights)
    #print("proba",proba)
    #print("gunT",gunT)
    #print("budoT",budoT)
    for i in range(gunT.shape[0]):
        for j in range(gunT.shape[1]):
            proba[i] = proba[i]+ gunT[i][j]*estm_weights[j]

    for i in range(budoT.shape[0]):
        for j in range(budoT.shape[1]):
            proba[gunT.shape[0]+i] = proba[gunT.shape[0]+i]+ budoT[i][j]* estm_weights[j]
    #print("proba:",proba)
    return Find_Optimal_Cutoff(real,proba)

def get_roc_auc_score(estm_weights,gunT,budoT):
    gunT = np.array(gunT)
    budoT = np.array(budoT)
    real_gun = np.zeros((gunT.shape[0],1))
    real_budo = np.zeros((budoT.shape[0],1))
    real_budo = real_budo+1
    real = np.vstack([real_gun,real_budo])
    proba = np.zeros((gunT.shape[0]+budoT.shape[0],1))
    proba = np.array(proba)

    for i in range(gunT.shape[0]):
        for j in range(gunT.shape[1]):
            proba[i] = proba[i]+ gunT[i][j]*estm_weights[j]

    for i in range(budoT.shape[0]):
        for j in range(budoT.shape[1]):
            proba[gunT.shape[0]+i] = proba[gunT.shape[0]+i]+ budoT[i][j]* estm_weights[j]
    return roc_auc_score(real,proba)

def normalize_weight(weight):
    total = 0
    for w in weight:
        total+=w
    
    for w in range(weight.shape[0]):
        weight[w] /= total
    return weight
        

def AUROC_fit(n_estimators,dataset,max_epochs=100,learning_rate = 0.01,limit = 0.0005):
    optimal_estm_weights = np.zeros((n_estimators,1))
    estm_weights = np.zeros((n_estimators,1))
    gun_num = 450
    budo_num = 450
    for i in range(n_estimators):
        estm_weights[i] = 1/n_estimators
        optimal_estm_weights[i] = 1/n_estimators
        #dataset = pd.read_excel('AUROC2.xlsm')
        #print(dataset)
    gun = dataset.iloc[:gun_num, 0:n_estimators]
    print(gun.shape)
    exit(1)
    gun = np.array(gun.T)

    budo = dataset.iloc[gun_num:budo_num+gun_num,0:n_estimators]
    budo = np.array(budo.T)
    budo_average=np.zeros((n_estimators,1))
    gun_average=np.zeros((n_estimators,1))
    budo_cov = np.cov(budo)
    gun_cov = np.cov(gun)
    for i in range(n_estimators):
        gun_average[i]=gmean(gun[i,:])

    for i in range(n_estimators):
        budo_average[i]=gmean(budo[i,:])
    budo_gun_average = budo_average - gun_average
    print(gun.shape)
    exit(1)
    max_auroc = get_roc_auc_score(estm_weights,gun.T,budo.T)
    budo_gun_cov = ((gun_num-1)*gun_cov + (budo_num-1)*budo_cov)/ (budo_num+gun_num-2)
    epo = 0
    while True:
        WTU = np.dot(estm_weights.T, budo_gun_average)
        
        WTSW = np.dot(np.dot(estm_weights.T,budo_gun_cov),estm_weights)
        SQRT_WTSW = WTSW**(1/2)

        SW = np.dot(budo_gun_cov,estm_weights)

        expression_1 = (-1/((3.14*2)**(1/2)))*(np.exp(-0.5*((WTU/SQRT_WTSW)**2) ))

        expression_2 = np.dot(SQRT_WTSW,budo_gun_average.T)/WTSW

        expression_3 = WTU/SQRT_WTSW*SW/WTSW

        expression_4 = expression_1*(expression_2.T - expression_3)

        estm_weights = estm_weights-(learning_rate*expression_4)
        
        estm_weights = normalize_weight(estm_weights)
        
        
        
        if max_auroc < get_roc_auc_score(estm_weights,gun.T,budo.T):
            max_auroc = get_roc_auc_score(estm_weights,gun.T,budo.T)
            optimal_estm_weights = estm_weights
            print("best auroc : ",max_auroc)
        epo +=1
        #print(epo)
        if max_epochs <= epo or SQRT_WTSW < limit :
            print("best epochs : ", epo)
            print("best auroc : ",max_auroc)
            print("optimal_weights",optimal_estm_weights)
            print("geoacc : ",get_geoacc(optimal_estm_weights,gun.T,budo.T))
            break
    

In [3]:
learning_rate = 0.01
limit = 0.0005
max_epoch = 600
dataset = pd.read_excel('AUROC2.xlsm')
AUROC_fit(20,dataset,max_epoch,learning_rate, limit)

(450, 20)
(20, 450)
best auroc :  0.9353333333333333
best auroc :  0.937837037037037
best auroc :  0.9395358024691358
best auroc :  0.9410617283950617
best auroc :  0.942153086419753
best auroc :  0.9433037037037038
best auroc :  0.9442518518518519
best auroc :  0.9452395061728395
best auroc :  0.9458814814814815
best auroc :  0.9465234567901235
best auroc :  0.9470024691358024
best auroc :  0.9476839506172838
best auroc :  0.9480938271604937


KeyboardInterrupt: 